# NBA FT GAME DIFFERENCE
Space to test whether making FTs makes a difference

In [37]:
!pip install pyarrow
#Imports
import pandas as pd
import pyarrow
import sklearn

You should consider upgrading via the 'c:\users\jason.huang\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [137]:
#Read Data
nba_2021 = pd.read_csv(r'C:\Users\jason.huang\Documents\Personal\DS\nba\nba_2021_stats.csv', encoding = "ISO-8859-1")
#CONVERT DATE TO STRING
nba_2021['GAME_DATE'] = nba_2021['GAME_DATE'].astype(str)

In [138]:
#Column Prep
#Add OPPOSITE TEAM
nba_2021["OPPOSITE_TEAM"] = nba_2021["MATCH_UP"].str[-3:]
#ADD TEAM GAME KEY
nba_2021["TEAM_GAME_KEY"] = nba_2021["TEAM"] + nba_2021["GAME_DATE"]
#ADD OPPOSITE TEAM GAME KEY
nba_2021["OPP_TEAM_GAME_KEY"] = nba_2021["OPPOSITE_TEAM"] + nba_2021["GAME_DATE"]
#ADD POINTS FROM MISSED FT
nba_2021["UPDATED_SCORE"] = (nba_2021["FTA"] -  nba_2021["FTM"] ) + nba_2021["PTS"] 

In [139]:
#Get Opp Team Data
nba_opp_team = nba_2021[['UPDATED_SCORE', 'TEAM_GAME_KEY']]

In [140]:
#Index Match
nba_merge = pd.merge(nba_2021, nba_opp_team, left_on='OPP_TEAM_GAME_KEY', right_on='TEAM_GAME_KEY', how='inner')
#Rename
nba_merge.rename(columns={'UPDATED_SCORE_x': 'UPDATED_SCORE','UPDATED_SCORE_y': 'UPDATED_OPP_TEAM_SCORE'}, inplace=True)

In [141]:
def update_outcome(row):
    if row['UPDATED_SCORE'] > row['UPDATED_OPP_TEAM_SCORE']:
        val = 'W'
    elif row['UPDATED_SCORE'] < row['UPDATED_OPP_TEAM_SCORE']:
        val = 'L'
    else:
        val = row['OUTCOME']
    return val
nba_merge['UPDATED_OUTCOME'] = nba_merge.apply(update_outcome, axis=1)

In [142]:
final = nba_merge[['TEAM', 'MATCH_UP', 'OPPOSITE_TEAM', 'GAME_DATE', 'OUTCOME', 'UPDATED_OUTCOME', 'PTS', 'UPDATED_SCORE', 'FTA', 'FTM', 'UPDATED_OPP_TEAM_SCORE']]

In [143]:
w_l_orig=final.groupby('TEAM')['OUTCOME'].apply(lambda x: (x=='W').sum()).reset_index(name='ORIG_W')
w_l_orig.sort_values(by=['ORIG_W'], ascending=False)

,TEAM,ORIG_W
28,UTA,52
23,PHX,51
22,PHI,49
1,BKN,48
12,LAC,47
7,DEN,47
16,MIL,46
24,POR,42
6,DAL,42
13,LAL,42


In [144]:
w_l_updated=final.groupby('TEAM')['UPDATED_OUTCOME'].apply(lambda x: (x=='W').sum()).reset_index(name='UPDATED_W')
w_l_updated.sort_values(by=['UPDATED_W'], ascending=False)

,TEAM,UPDATED_W
22,PHI,52
28,UTA,51
23,PHX,50
1,BKN,47
7,DEN,46
12,LAC,45
16,MIL,45
0,ATL,43
13,LAL,43
6,DAL,41


In [145]:
data = [['GSW', 'West'], ['PHX', 'West'], ['UTA', 'West'], ['UTA', 'West'], ['DAL', 'West'], ['LAC', 'West'], ['DEN', 'West'], ['MEM', 'West'], ['LAL', 'West'], ['POR', 'West'], ['OKC', 'West'], ['SAC', 'West'], ['SAS', 'West'], ['MIN', 'West'], ['NOP', 'West'], ['HOU', 'West'], ['BKN', 'East'], ['MIA', 'East'], ['CHI', 'East'], ['CLE', 'East'], ['CHA', 'East'], ['TOR', 'East'], ['PHI', 'East'], ['NYK', 'East'], ['WAS', 'East'], ['MIL', 'East'], ['IND', 'East'], ['BOS', 'East'], ['ATL', 'East'], ['ORL', 'East'], ['DET', 'East']]

team_conf = pd.DataFrame(data, columns=['TEAM', 'CONFERENCE'])
team_conf

,TEAM,CONFERENCE
0,GSW,West
1,PHX,West
2,UTA,West
3,UTA,West
4,DAL,West
5,LAC,West
6,DEN,West
7,MEM,West
8,LAL,West
9,POR,West


In [146]:
final_merge = pd.merge(w_l_orig, w_l_updated, on='TEAM', how='inner')
final_merge['DIFF_W'] = final_merge['UPDATED_W'] - final_merge['ORIG_W']

final_merge_conf = pd.merge(final_merge, team_conf, on='TEAM', how='inner')
final_merge_conf.sort_values(by=['CONFERENCE','UPDATED_W'], ascending=False)

,TEAM,ORIG_W,UPDATED_W,DIFF_W,CONFERENCE
28,UTA,52,51,-1,West
29,UTA,52,51,-1,West
23,PHX,51,50,-1,West
7,DEN,47,46,-1,West
12,LAC,47,45,-2,West
13,LAL,42,43,1,West
6,DAL,42,41,-1,West
24,POR,42,40,-2,West
14,MEM,38,39,1,West
9,GSW,39,36,-3,West
